### Input: CIF
- Must specify central atom wyckoff position and radius of cluster

### Output: xyz

In [32]:
import os
from pymatgen.core.structure import Structure
from pymatgen.core.structure import Molecule
from pymatgen.io.xyz import XYZ
import numpy as np

center_wk_pos = 0 # Atom to put at center of cluster
radius = 3 # Radius of cluster

def get_cluster_around_wyckoff_position(structure, wyckoff_number, cluster_radius=2.5):
    """
    Extracts a cluster around a specified Wyckoff position in a crystal structure.

    Parameters:
    - structure: pymatgen Structure object
    - wyckoff_position: str, Wyckoff position to center the cluster around
    - cluster_radius: float, radius of the cluster in angstroms

    Returns:
    - pymatgen Structure object representing the extracted cluster
    """
    structure_analyzer = SpacegroupAnalyzer(structure)
    symmetrized_structure = structure_analyzer.get_symmetrized_structure()

    wyckoff_cords = symmetrized_structure.sites[wyckoff_number].coords
    
    # Get the Wyckoff positions that match the specified Wyckoff position
    sites = symmetrized_structure.get_sites_in_sphere(symmetrized_structure.cart_coords[wyckoff_number],
                                                              cluster_radius)

    cluster_structure = Structure.from_sites(sites)

    return cluster_structure


def create_xyz_from_cif(cif_filename, out_xyz_filename, center_wk_pos, radius):
    structure = Structure.from_file(cif_filename)
    cluster_structure = get_cluster_around_wyckoff_position(structure, center_wk_pos, radius)
    cluster_molecule = Molecule.from_sites(cluster_structure)
    XYZ(cluster_molecule).write_file(out_xyz_filename)

# Get a list of all files in the current directory
file_list = [file for file in os.listdir() if file.endswith(".cif")]

# Loop through each file in the folder
for cif_path in file_list:
    
    xyz_path = cif_path.replace(".cif",".xyz")
    create_xyz_from_cif(cif_path, xyz_path, center_wk_pos=center_wk_pos, radius=radius)